# Build a song recommender system

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# Load some music data

In [8]:
song_data =pd.read_csv('Dataset/song_data.csv')

In [36]:
song_data

,user_id,song_id,listen_count,title,artist,song
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Jack Johnson,The Cove - Jack Johnson
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Paco De Lucia,Entre Dos Aguas - Paco De Lucia
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Kanye West,Stronger - Kanye West
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,Jack Johnson,Constellations - Jack Johnson
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,Foo Fighters,Learn To Fly - Foo Fighters
...,...,...,...,...,...,...
1116604,276e43ad698705e5011e5091f367d951b21246f5,SOWTYVO12AB0180BDA,8,Surf Wax America,Weezer,Surf Wax America - Weezer
1116605,276e43ad698705e5011e5091f367d951b21246f5,SOWZHOY12A6701F9FC,1,Kill The King,Megadeth,Kill The King - Megadeth
1116606,276e43ad698705e5011e5091f367d951b21246f5,SOXJOZW12A6701E90E,7,You Think I Ain't Worth A Dollar_ But I Feel L...,Queens Of The Stone Age,You Think I Ain't Worth A Dollar_ But I Feel L...
1116607,276e43ad698705e5011e5091f367d951b21246f5,SOXNKNC12A6701F7A8,1,Superbeast,Rob Zombie,Superbeast - Rob Zombie


In [37]:
len(song_data)

1116609

In [34]:
song_data.describe()

,listen_count
count,1.116609e+06
mean,3.290769e+00
std,7.202534e+00
min,1.000000e+00
25%,1.000000e+00
50%,1.000000e+00
75%,3.000000e+00
max,9.200000e+02


## Show the most popular songs in the dataset

In [21]:
#song_data['song'].show()
popular_song=pd.DataFrame(song_data['song'].value_counts().items(), columns=['song', 'count'])

In [29]:
popular_song

,song,count
0,Sehr kosmisch - Harmonia,5970
1,Undo - Björk,5281
2,You're The One - Dwight Yoakam,4806
3,Dog Days Are Over (Radio Edit) - Florence + Th...,4536
4,Revelry - Kings Of Leon,4339
...,...,...
9947,Accidntel Deth (Album Version) - Rilo Kiley,16
9948,Marching Theme - Neutral Milk Hotel,15
9949,Trahison - Vitalic,15
9950,Hubcap - Sleater-kinney,12


In [28]:
popular_song.head(10)

,song,count
0,Sehr kosmisch - Harmonia,5970
1,Undo - Björk,5281
2,You're The One - Dwight Yoakam,4806
3,Dog Days Are Over (Radio Edit) - Florence + Th...,4536
4,Revelry - Kings Of Leon,4339
5,Horn Concerto No. 4 in E flat K495: II. Romanc...,3949
6,Secrets - OneRepublic,3916
7,Tive Sim - Cartola,3185
8,Fireflies - Charttraxx Karaoke,3171
9,Hey_ Soul Sister - Train,3132


# Count the number of unique users in the data

In [31]:
users = song_data['user_id'].unique()

In [33]:
len(users)

66346

In [38]:
songs = song_data['song'].unique()

In [39]:
len(songs)

9952

# Create a song recommender

In [48]:
from sklearn.model_selection import train_test_split

In [41]:
train_data, test_data = train_test_split(song_data, test_size = 0.20, random_state=0)

In [55]:
train_data

,user_id,song_id,listen_count,title,artist,song
308223,4c1ea8245b67c2862e12f02b78e345cc7bdd7282,SOWVRVI12A6701E0B4,7,Outshined,Soundgarden,Outshined - Soundgarden
308537,6df2dd5a3bb9042da67ab82baf0727bca10ec05e,SOMEUED12A6701DBEA,1,Pump It,Black Eyed Peas,Pump It - Black Eyed Peas
234078,fd84fea1c5df235d73ebd3cf4a8c638ca1f3cfdb,SOBMCEI12A6D4F65BD,2,My Lovin' (You're Never Gonna Get It) (LP Vers...,En Vogue,My Lovin' (You're Never Gonna Get It) (LP Vers...
391502,d30183edf03302cdc25cb39798ae50a4395fa50f,SOCJAMC12A8C13F245,2,Precious Things,Tori Amos,Precious Things - Tori Amos
379458,9d4dfbd3614c1a4caab5f7fda723563befa626e8,SOJOIKI12AB0187525,1,Hero (Album Version),Skillet,Hero (Album Version) - Skillet
...,...,...,...,...,...,...
359783,243ca9574539ba638847f407e22ce0c3953ad391,SONUOOC12A8C136211,3,Heartbeat,Nneka,Heartbeat - Nneka
152315,35c63042c895bc5b198cab8e04b718469b7543ba,SOGHOUY12A8C143CB9,4,Dinosaur,Ke$ha,Dinosaur - Ke$ha
963395,109c70af3471629c753e693e741f805024b85b7d,SOKVZHI12A58A7A13E,5,Everywhere I Go,Hollywood Undead,Everywhere I Go - Hollywood Undead
117952,0de9ced79f560aa0e51c413fc54ec5d39f1e7386,SOPBFHP12A8C131F30,30,Here_ There And Everywhere,Erin Bode,Here_ There And Everywhere - Erin Bode


In [54]:
test_data

,user_id,song_id,listen_count,title,artist,song
333951,8f8f909dd86e77969d1f8753e10ac56c4c68aa72,SOZXQRQ12A6701C323,2,Up Where We Belong,Joe Cocker / Jennifer Warnes,Up Where We Belong - Joe Cocker / Jennifer Warnes
492194,d5a1b1b8142327cea378f262b96ef5675c8bff99,SOZKNSB12A8C140F11,6,Replay,Iyaz,Replay - Iyaz
525465,58907ab6047361c85eb92be39dcbd2f0eb924684,SOLFBXX12A8AE47A51,1,Surfacing (Live version) (Album Version),Slipknot,Surfacing (Live version) (Album Version) - Sli...
465962,f22c0aa5308322685a42fc10fa05ea3f4a6e96b4,SOWTAVD12B0B8090C2,1,Just Like You Imagined,Nine Inch Nails,Just Like You Imagined - Nine Inch Nails
162333,3134803ece9906f87bb1b64a63058ef26c603336,SOCCEFN12A6701C548,1,Tonight (Featuring Mobb Deep & Joe Hooker)(Alb...,The Notorious B.I.G.,Tonight (Featuring Mobb Deep & Joe Hooker)(Alb...
...,...,...,...,...,...,...
15962,22b6baffe7de156c64060ebee5b3618f2aaf8c89,SOVWADY12AB0189C63,2,Party In The U.S.A.,Miley Cyrus,Party In The U.S.A. - Miley Cyrus
62745,4fd45ecbcc53725aa081527904bcfa166eb4e2c1,SOSACAB12AB01876E4,1,From The Station,Soltero,From The Station - Soltero
276130,53778a8a259574fb76b3252efb8b47e5aac5ae7a,SOFZSAT12AF72A0806,7,Youth Against Fascism,Sonic Youth,Youth Against Fascism - Sonic Youth
535667,bbcc1ebf3046eedfb94c89787b6500e8738964e4,SOIZFTE12AB0186842,1,Fuck Kitty,Frumpies,Fuck Kitty - Frumpies


## Create a very simple popularity recommender

In [57]:
import Recommenders as Recommenders
pm = Recommenders.popularity_recommender_py()
pm.create(train_data, 'user_id', 'song')

## Use the popularity model to make some predictions


In [59]:
pm.recommend(users[0])

,user_id,song,score,Rank
7128,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Sehr kosmisch - Harmonia,4812,1.0
9083,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Undo - Björk,4211,2.0
9879,b80344d063b5ccb3212f76538f3d9e43d87dca9e,You're The One - Dwight Yoakam,3819,3.0
2068,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Dog Days Are Over (Radio Edit) - Florence + Th...,3624,4.0
6775,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Revelry - Kings Of Leon,3485,5.0
3614,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Horn Concerto No. 4 in E flat K495: II. Romanc...,3188,6.0
7116,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Secrets - OneRepublic,3103,7.0
8846,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Tive Sim - Cartola,2551,8.0
2718,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Fireflies - Charttraxx Karaoke,2525,9.0
3486,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Hey_ Soul Sister - Train,2505,10.0


In [62]:
pm.recommend(users[2])

,user_id,song,score,Rank
7128,bd4c6e843f00bd476847fb75c47b4fb430a06856,Sehr kosmisch - Harmonia,4812,1.0
9083,bd4c6e843f00bd476847fb75c47b4fb430a06856,Undo - Björk,4211,2.0
9879,bd4c6e843f00bd476847fb75c47b4fb430a06856,You're The One - Dwight Yoakam,3819,3.0
2068,bd4c6e843f00bd476847fb75c47b4fb430a06856,Dog Days Are Over (Radio Edit) - Florence + Th...,3624,4.0
6775,bd4c6e843f00bd476847fb75c47b4fb430a06856,Revelry - Kings Of Leon,3485,5.0
3614,bd4c6e843f00bd476847fb75c47b4fb430a06856,Horn Concerto No. 4 in E flat K495: II. Romanc...,3188,6.0
7116,bd4c6e843f00bd476847fb75c47b4fb430a06856,Secrets - OneRepublic,3103,7.0
8846,bd4c6e843f00bd476847fb75c47b4fb430a06856,Tive Sim - Cartola,2551,8.0
2718,bd4c6e843f00bd476847fb75c47b4fb430a06856,Fireflies - Charttraxx Karaoke,2525,9.0
3486,bd4c6e843f00bd476847fb75c47b4fb430a06856,Hey_ Soul Sister - Train,2505,10.0
